In [1]:
from aoc2023 import day1
from aoc2023 import maps
import numpy as np
import functools

In [41]:
data = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

In [67]:
data = day1.read_data_string("input.txt")
# data = day1.read_data_string("input_sample.txt")
lines = day1.split_data(data) 
paragraphs = [day1.split_data_p(paragraph) for paragraph in day1.split_data_paragraphs(data)]

In [68]:
rules = paragraphs[0]
pieces = paragraphs[1]

In [69]:
parse_rules = {}
for rule in rules: 
    code, expressions = rule.replace("}", "").split("{")
    parse_rules[code] = expressions.split(",")

In [70]:
parse_rules

{'jz': ['m<1130:R', 'm>1525:A', 'R'],
 'cb': ['m<1514:R', 's<791:A', 'a>1449:R', 'xmm'],
 'cn': ['x<3563:R', 'R'],
 'qb': ['m>1908:A', 'a>1204:xmt', 'x>1001:A', 'zgl'],
 'dr': ['m<3408:A', 'a>1459:A', 'x>1500:R', 'R'],
 'nfv': ['a<1625:bbz', 'A'],
 'zzt': ['s>3266:R', 's<2765:A', 'R'],
 'tk': ['m<852:gkx', 's<2289:fh', 'A'],
 'qc': ['a<1680:R', 'R'],
 'klm': ['x<1453:R', 'm<2557:A', 'vms'],
 'zc': ['x>2239:sqj', 'a<2898:cch', 's>3063:xjt', 'sdq'],
 'pdn': ['s>626:A', 'm>982:A', 'R'],
 'ld': ['s>1514:qj', 'a>1307:lff', 'jjf'],
 'rtt': ['x>377:R', 'm<2066:R', 'R'],
 'njn': ['a<3023:R', 'R'],
 'bvp': ['x<299:A', 'R'],
 'nlt': ['m>1065:bcz', 'm>537:kz', 'fdd'],
 'mvx': ['m>3404:R', 'R'],
 'hgv': ['x<842:jh', 'A'],
 'slt': ['s>1572:A', 's<1560:R', 's<1564:A', 'A'],
 'sc': ['s<212:klm', 'm<2020:dm', 'dj'],
 'pk': ['a<3727:A', 'm>3094:A', 's<2200:A', 'A'],
 'tm': ['s>2860:R', 'A'],
 'vgc': ['a>1682:R', 's<1630:A', 's>1790:R', 'A'],
 'bcz': ['s>1401:R', 'R'],
 'zzn': ['a>3467:tmz', 'x<1855:tdx

In [71]:
# --CAKE FOOTPRINT-- Please do not remove this comment even if you modify the code snippet, it is needed for traceability purposes

def parse_string_to_dict(input_string):
    # Remove the curly braces and split the string into pairs
    pairs = input_string.strip('{}').split(',')
    
    # Initialize an empty dictionary
    parsed_dict = {}
    
    # Split each pair and add to the dictionary
    for pair in pairs:
        key, value = pair.split('=')
        parsed_dict[key] = int(value)
        
    return parsed_dict



In [72]:
p1 = 0 
for piece in pieces: 
    dict_piece = parse_string_to_dict(piece)
    end = False
    rule = "in"
    while not end: 
        
        for c in parse_rules[rule]: 
            if ":" in c: 
                exp_c, t_c = c.split(":")
                if ">" in exp_c: 
                    k, v = exp_c.split(">")
                    if dict_piece[k] > int(v): 
                        rule = t_c 
                        break
                elif "<" in exp_c: 
                    k, v = exp_c.split("<")
                    if dict_piece[k] < int(v): 
                        rule = t_c 
                        break
                
                else: 
                    raise ValueError(exp_c)
            else: 
                rule = c

        if rule == "A": 
            accepted = True
            end = True
        if rule == "R": 
            accepted = False
            end = True

    if accepted: 
        sum_vals = 0 
        for key, value in dict_piece.items(): 
            sum_vals += value
        p1 += sum_vals

In [73]:
xmas_max = {'x': 4000, 'm': 4000, 'a': 4000, 's': 4000}
xmas_min = {'x': 0, 'm': 0, 'a': 0, 's': 0}

In [74]:
def get_interval(rule, xmas_max, xmas_min): 
    # print(rule)
    if rule == "A": 
        prod = 1
        for l in "xmas": 
            prod *= xmas_max[l] - xmas_min[l]
        return prod
    
    if rule == "R": 
        return 0
    
    sum_rules = 0 
    for c in parse_rules[rule]: 
        if ":" in c:
            xmas_max_in = xmas_max.copy()
            xmas_min_in = xmas_min.copy()
            exp_c, t_c = c.split(":")
            if ">" in exp_c: 
                k, v = exp_c.split(">")
                v = int(v)
                xmas_max[k] = v 
                xmas_min_in[k] = v 
                sum_rules += get_interval(t_c, xmas_max_in, xmas_min_in)


            elif "<" in exp_c: 
                k, v = exp_c.split("<")
                v = int(v)
                xmas_min[k] = v -1
                xmas_max_in[k] = v -1  
                sum_rules += get_interval(t_c, xmas_max_in, xmas_min_in)

            else: 
                raise ValueError(exp_c)
        else: 
            xmas_max_in = xmas_max.copy()
            xmas_min_in = xmas_min.copy()
            sum_rules += get_interval(c, xmas_max_in, xmas_min_in)
    return sum_rules

In [75]:
rule = "in"
xmas_max = {'x': 4000, 'm': 4000, 'a': 4000, 's': 4000}
xmas_min = {'x': 0, 'm': 0, 'a': 0, 's': 0}
p2 = get_interval(rule, xmas_max, xmas_min)

In [76]:
p2

143760172569135

In [19]:
for key, value in parse_string_to_dict(piece).items(): 
    print(value)

2127
1623
2188
1013
